In [53]:
import pandas as pd
from datetime import datetime
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
from tqdm import tnrange, tqdm_notebook, tqdm
import glob
import numpy as np

currency = "bitcoin"
symbol = "BTC"
tweets_clean_file = f'../data/twitter/{symbol}/{currency}_tweets_clean.csv'
path = f'../data/twitter/{symbol}'

In [54]:
df_clean = pd.read_csv(tweets_clean_file)
print(df_clean.shape)
df_clean.head(5)

(93, 7)


ID                                               Text  \
0  1480507769610510343  Legendary investor Bill Miller is still buying...   
1  1480190307224465408                               Stay hungry. Bitcoin   
2  1480259013455400961  1/ BTC 🤝 ICP direct integration (🚫🌉)cryptograp...   
3  1480722463679860736  RT : Gold has gone from 0 to $1850 in 5000 yea...   
4  1480722458931970050  RT : It's amazing, this is a project I've been...   

             UserName  UserFollowerCount  RetweetCount  Likes  \
0    Bitcoin Magazine            1921879           648   3898   
1    Michael Saylor⚡️            2084920          2017  21352   
2  DFINITY Foundation             463883           448   1173   
3          Mr.BUIDL🦇🔊                503           607      0   
4       Fahrul_Oiiker                  3             1      0   

                        CreatedAt  
0  Mon Jan 10 11:52:13 +0000 2022  
1  Sun Jan 09 14:50:44 +0000 2022  
2  Sun Jan 09 19:23:45 +0000 2022  
3  Tue Jan 11 02:05:20 +0000 2022  
4  Tue Jan 11 02:05:19 +0000 2022

In [55]:
df_clean = df_clean.sort_values(by='ID') # the bigger the ID, the most recent the tweet 

In [56]:
analyzer = SentimentIntensityAnalyzer()
compound = []
for i,s in enumerate(tqdm(df_clean['Text'])):
    vs = analyzer.polarity_scores(s)
    compound.append(vs["compound"])
df_clean["compound"] = compound
df_clean.head(2)

100%|██████████| 93/93 [00:00<00:00, 3917.12it/s]


ID                                               Text  \
1  1480190307224465408                               Stay hungry. Bitcoin   
2  1480259013455400961  1/ BTC 🤝 ICP direct integration (🚫🌉)cryptograp...   

             UserName  UserFollowerCount  RetweetCount  Likes  \
1    Michael Saylor⚡️            2084920          2017  21352   
2  DFINITY Foundation             463883           448   1173   

                        CreatedAt  compound  
1  Sun Jan 09 14:50:44 +0000 2022    0.0000  
2  Sun Jan 09 19:23:45 +0000 2022    0.3818

In [57]:
scores = []
for i, s in tqdm(df_clean.iterrows(), total=df_clean.shape[0]):
    scores.append(float(s["compound"]) * (float(s["UserFollowerCount"])+1) * (float(s["Likes"])+1))
df_clean["score"] = scores
df_clean.head(2)

100%|██████████| 93/93 [00:00<00:00, 5942.12it/s]


ID                                               Text  \
1  1480190307224465408                               Stay hungry. Bitcoin   
2  1480259013455400961  1/ BTC 🤝 ICP direct integration (🚫🌉)cryptograp...   

             UserName  UserFollowerCount  RetweetCount  Likes  \
1    Michael Saylor⚡️            2084920          2017  21352   
2  DFINITY Foundation             463883           448   1173   

                        CreatedAt  compound         score  
1  Sun Jan 09 14:50:44 +0000 2022    0.0000  0.000000e+00  
2  Sun Jan 09 19:23:45 +0000 2022    0.3818  2.079282e+08

In [58]:
n = 20000  #chunk row size
chunks_df = [df_clean[i:i+n] for i in range(0,df_clean.shape[0],n)]

sep_char = '~'
for chunk_df in chunks_df:
    chunk_min = chunk_df['ID'].min()
    chunk_max = chunk_df['ID'].max()
    date_from = (datetime.strptime(chunk_df.iloc[0]['CreatedAt'], '%a %b %d %X %z %Y')).strftime('%Y-%m-%d %H-%M-%S')
    date_to = (datetime.strptime(chunk_df.iloc[-1]['CreatedAt'], '%a %b %d %X %z %Y')).strftime('%Y-%m-%d %H-%M-%S')
    print(date_from, date_to)

    # Write into csv
    chunk_df.to_csv(f"{path}/{date_from}{sep_char}{date_to}.csv", header=True, index=False)

2022-01-09 14-50-44 2022-01-11 02-05-20


In [59]:
ENVS = ['CRYPTO', 'LINE_COUNT', 'MOST_RECENT_FILE', 'MOST_RECENT_ID'] # Stored in var.csv

def get_var(key, crypto):
    df_var = pd.read_csv("data/twitter/var.csv", sep=',',
                         dtype={'LINE_COUNT': np.int32})
    return df_var[key].loc[df_var['CRYPTO'] == crypto].values[0]

def update_var(key, value, crypto):
    df_var = pd.read_csv("data/twitter/var.csv", sep=',',
                         dtype={'LINE_COUNT': np.int32})
    df_var[key].loc[df_var['CRYPTO'] == crypto] = str(value)
    df_var.to_csv("var.csv", index=False)
    
def add_new_crypto(crypto):
    df_var = pd.read_csv("data/twitter/var.csv", sep=',',
                         dtype={'LINE_COUNT': np.int32})
    if df_var[ENVS[0]].loc[df_var['CRYPTO'] == crypto].empty:
        new_line = pd.DataFrame([[crypto,-1,"",0]], columns=ENVS)
        df_var = df_var.append(new_line)
        df_var.to_csv("data/twitter/var.csv", index=False)

In [60]:
files = glob.glob(f"{path}/*~*.csv.csv")
files = sorted(files)
last_file = files[-1]
print(last_file)
last_df = pd.read_csv(last_file)
last_elem = last_df.tail(1)
print(last_elem['ID'])
print(last_df.shape)

add_new_crypto(CURRENCY_SYMBOL)
update_var(ENVS[1], last_df.shape[0], CURRENCY_SYMBOL)
update_var(ENVS[2], last_file, CURRENCY_SYMBOL)
update_var(ENVS[3], last_elem, CURRENCY_SYMBOL)

IndexError: list index out of range

In [ ]:
var_df = pd.read_csv('../data/twitter/var.csv')
var_df